# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,10.97,80,75,0.00,US,1724866869
1,1,adamstown,-25.0660,-130.1015,18.68,66,98,1.74,PN,1724866870
2,2,jamestown,42.0970,-79.2353,28.90,65,75,3.60,US,1724866859
3,3,nelson bay,-32.7167,152.1500,17.78,50,22,2.24,AU,1724866872
4,4,thunder bay,48.4001,-89.3168,17.95,59,75,6.69,CA,1724866774


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    line_color='black'
)

# Display the map
cities_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Create a copy of the above dataframe to avoid dropna error later
city_copy = city_data_df.copy()

# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_copy[(city_copy["Max Temp"] > 17) &\
    (city_copy["Max Temp"] < 23) &\
    (city_copy["Humidity"] < 70) &\
    (city_copy["Wind Speed"] < 4.5) &\
    (city_copy["Cloudiness"] < 50)\
    ]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,nelson bay,-32.7167,152.1500,17.78,50,22,2.24,AU,1724866872
71,71,chulakkurgan,43.7650,69.1814,17.35,54,8,0.47,KZ,1724866948
74,74,les avirons,-21.2418,55.3394,18.05,68,20,1.54,RE,1724866952
111,111,constantia,44.1833,28.6500,21.50,66,45,0.80,RO,1724866991
194,194,beroroha,-21.6667,45.1667,21.00,62,0,1.29,MG,1724867082
197,197,vernon,50.2581,-119.2691,18.85,44,22,2.06,CA,1724867085
230,230,byron bay,-28.6500,153.6167,19.49,57,0,2.10,AU,1724867124
234,234,leninskoye,41.7589,69.3864,18.73,40,0,3.10,KZ,1724867130
252,252,pendleton,45.6721,-118.7886,18.21,45,0,1.54,US,1724867152
263,263,tavsanli,39.5424,29.4987,19.87,57,19,3.11,TR,1724867164


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ["" for _ in range(len(hotel_df))]

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,nelson bay,AU,-32.7167,152.1500,50,
71,chulakkurgan,KZ,43.7650,69.1814,54,
74,les avirons,RE,-21.2418,55.3394,68,
111,constantia,RO,44.1833,28.6500,66,
194,beroroha,MG,-21.6667,45.1667,62,
197,vernon,CA,50.2581,-119.2691,44,
230,byron bay,AU,-28.6500,153.6167,57,
234,leninskoye,KZ,41.7589,69.3864,40,
252,pendleton,US,45.6721,-118.7886,45,
263,tavsanli,TR,39.5424,29.4987,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set base URL 
base_url = "https://api.geoapify.com/v2/places"

# Set the initially known parameter variables for the search
categories = "accommodation.hotel"
radius = 10000
limit = 20

filters = {}
bias = {}

# Set parameters to search for a hotel
params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nelson bay - nearest hotel: No hotel found
chulakkurgan - nearest hotel: No hotel found
les avirons - nearest hotel: Floralys
constantia - nearest hotel: Hotel Dali
beroroha - nearest hotel: No hotel found
vernon - nearest hotel: Okanagan Royal Park Inn
byron bay - nearest hotel: Lord Byron Resort
leninskoye - nearest hotel: No hotel found
pendleton - nearest hotel: Howard Johnson Pendleton
tavsanli - nearest hotel: No hotel found
ilgaz - nearest hotel: Ilgaz Ögretmenevi
sao vicente - nearest hotel: Pousada Vitória
saint-pierre - nearest hotel: Tropic Hotel
jackson - nearest hotel: Brookside
povarovo - nearest hotel: No hotel found
klyavlino - nearest hotel: No hotel found
shubarkuduk - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
kalodzishchy - nearest hotel: MonChalet


,City,Country,Lat,Lng,Humidity,Hotel Name
3,nelson bay,AU,-32.7167,152.1500,50,No hotel found
71,chulakkurgan,KZ,43.7650,69.1814,54,No hotel found
74,les avirons,RE,-21.2418,55.3394,68,Floralys
111,constantia,RO,44.1833,28.6500,66,Hotel Dali
194,beroroha,MG,-21.6667,45.1667,62,No hotel found
197,vernon,CA,50.2581,-119.2691,44,Okanagan Royal Park Inn
230,byron bay,AU,-28.6500,153.6167,57,Lord Byron Resort
234,leninskoye,KZ,41.7589,69.3864,40,No hotel found
252,pendleton,US,45.6721,-118.7886,45,Howard Johnson Pendleton
263,tavsanli,TR,39.5424,29.4987,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    line_color='black',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)